In [1]:
import pandas as pd
import numpy as np
import copy
import sys
import os
sys.path.append("../")
from parser.utils import load_json, dfs_cardinality, estimate_scan_in_mb, dumper
from models.feature.single_xgboost_feature import find_top_k_operators, featurize_one_plan, get_top_k_table_by_size
from utils.load_trace import load_trace, create_concurrency_dataset, load_trace_all_version
from models.concurrency.utils import pre_info_train_test_seperation
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from models.single.stage import SingleStage
from models.concurrency.complex_models import ConcurrentRNN
np.set_printoptions(suppress=True)

In [2]:
parsed_queries_path = "/Users/ziniuw/Desktop/research/Data/AWS_trace/mixed_aurora/aurora_mixed_parsed_queries.json"
#parsed_queries_path = "/Users/ziniuw/Desktop/research/Data/AWS_trace/mixed_postgres/postgres_mixed_parsed_queries.json"
plans = load_json(parsed_queries_path, namespace=False)
folder_name = "mixed_aurora"
directory = f"/Users/ziniuw/Desktop/research/Data/AWS_trace/{folder_name}/"
all_raw_trace, all_trace = load_trace_all_version(directory, 8, concat=True)
all_concurrency_df = []
for trace in all_trace:
    concurrency_df = create_concurrency_dataset(trace, engine=None, pre_exec_interval=200)
    all_concurrency_df.append(concurrency_df)
concurrency_df = pd.concat(all_concurrency_df, ignore_index=True)

In [3]:
#parsed_queries_path = "/Users/ziniuw/Desktop/research/Data/AWS_trace/long_redshift/mixed_redshift_parsed_queries.json"
parsed_queries_path = "/Users/ziniuw/Desktop/research/Data/AWS_trace/mixed_postgres/postgres_mixed_parsed_queries.json"
plans = load_json(parsed_queries_path, namespace=False)

directories = ["saved_results/postgres/archive", "saved_results/postgres/"]
all_trace = []
for directory in directories:
    for file in os.listdir(directory):
        if file.endswith(".csv"):
            file = os.path.join(directory, file)
            print(file)
            df = pd.read_csv(file)
            df = df[df['run_time_s'] > 0]
            if "error" in df.columns:
                df = df[df["error"] == False]
            all_trace.append(df)
all_concurrency_df = []
for trace in all_trace:
    concurrency_df = create_concurrency_dataset(trace, engine=None, pre_exec_interval=None)
    all_concurrency_df.append(concurrency_df)

concurrency_df = pd.concat(all_concurrency_df, ignore_index=True)

saved_results/postgres/archive/timeout_600_num_clients10_baseline.csv
saved_results/postgres/archive/postgres_10_clients_ours_0.csv
saved_results/postgres/archive/timeout_600_num_clients4_baseline.csv
saved_results/postgres/archive/clients_8_timeout_1000_baseline_timeout_1000_ours_2.csv
saved_results/postgres/archive/clients_8_timeout_1000_baseline_timeout_1000_ours.csv
saved_results/postgres/archive/postgres_4_clients_baseline_0.csv
saved_results/postgres/archive/timeout_600_num_clients10_ours.csv
saved_results/postgres/archive/timeout_600_num_clients2_baseline.csv
saved_results/postgres/archive/postgres_6_clients_ours_0.csv
saved_results/postgres/archive/postgres_1453912639619907921_baseline_0.csv
saved_results/postgres/archive/clients_4_timeout_400_baseline_timeout_400_ours_2.csv
saved_results/postgres/archive/postgres_6_clients_baseline_0.csv
saved_results/postgres/archive/snowset_1453912639619907921_postgres_replay2_timeout_400_ours_3.csv
saved_results/postgres/archive/snowset_145

In [4]:
train_trace_df_sep, eval_trace_df_sep = pre_info_train_test_seperation(concurrency_df)
print(len(train_trace_df_sep), len(eval_trace_df_sep))
np.random.seed(0)
train_idx = np.random.choice(len(concurrency_df), size=int(0.8 * len(concurrency_df)), replace=False)
test_idx = [i for i in range(len(concurrency_df)) if i not in train_idx]
train_trace_df = copy.deepcopy(concurrency_df.iloc[train_idx])
eval_trace_df = concurrency_df.iloc[test_idx]
#eval_trace_df = copy.deepcopy(eval_trace_df[eval_trace_df['num_concurrent_queries'] > 0])
print(len(train_trace_df), len(eval_trace_df))

64565 34158
107119 19644


In [5]:
mean = []
std = []
for i, rows in concurrency_df.groupby("query_idx"):
    runtime = rows["runtime"].values
    print(i, len(rows), np.mean(runtime), np.min(runtime), np.max(runtime), np.std(runtime))
    mean.append(np.mean(runtime))
    std.append(np.std(runtime))
mean = np.asarray(mean)
std = np.asarray(std)

0.0 603 4.659716329764371 1.1410207748413086 53.739098072052 6.550661489155383
1.0 542 0.14275977532362144 0.0263817310333251 34.41116452217102 1.507582940657969
2.0 564 127.44124870731476 0.8265385627746582 585.935792684555 95.95257033129505
3.0 1005 372.73086169513306 2.7054457664489746 1011.4952692985536 200.39576165306025
4.0 437 321.68464606304735 0.0244207382202148 1000.0734148025512 218.02699410848962
5.0 331 141.93918162890432 12.332037448883057 740.7330555915833 92.20249152028772
6.0 419 152.9720044596952 11.727198362350464 582.469349861145 101.78804757751902
7.0 613 2.3078964266069173 0.5350992679595947 39.29085445404053 3.817908125696016
8.0 808 4.9024348515685245 0.143965482711792 141.49800443649292 12.145054830924266
9.0 370 0.756486685211594 0.0420949459075927 20.87054872512817 2.1883416331101877
10.0 383 0.3444103061685985 0.0350487232208251 43.343369245529175 2.212481563221963
11.0 921 18.98686974625893 0.8305304050445557 213.36397647857663 26.477645358711328
12.0 416 1

In [6]:
idx = np.where(std > 5)[0]
len(idx)

206

In [5]:
ss = SingleStage(use_table_features=True, true_card=False)
#df = ss.featurize_data(train_trace_df, parsed_queries_path)
#parsed_queries_path = "/Users/ziniuw/Desktop/research/Data/AWS_trace/long_redshift/mixed_redshift_parsed_queries.json"
df = ss.featurize_data(concurrency_df, parsed_queries_path)
ss.train(df)

Top 20 operators contains 0.9642989959092599 total operators


In [ ]:
rnn = ConcurrentRNN(ss, 
                    "postgres",
                    input_size=len(ss.all_feature[0]) * 2 + 7,
                    embedding_dim=128,
                    hidden_size=256,
                    num_layers=2,
                    loss_function="q_loss",
                    last_output=True,
                    use_separation=False,
                    ignore_short_running=True,
                    short_running_threshold=5
                   )
rnn.train(train_trace_df, eval_trace_df, lr=0.001, loss_function="q_loss", val_on_test=True)
#rnn.load_model("checkpoints")

********Epoch 0, training loss: 100.37991619434123 || evaluation loss: ********


100%|█████████████████████████████████████████████████████████████████████████| 154/154 [00:03<00:00, 39.29it/s]


50% absolute error is 4.7589781284332275, q-error is 1.4237536191940308
90% absolute error is 36.594293212890626, q-error is 3.607741832733156
95% absolute error is 54.92321739196774, q-error is 5.852267122268666
********Epoch 5, training loss: 6.156332835804319 || evaluation loss: ********


100%|█████████████████████████████████████████████████████████████████████████| 154/154 [00:04<00:00, 37.73it/s]


50% absolute error is 4.157587289810181, q-error is 1.2878175973892212
90% absolute error is 34.55337600708009, q-error is 3.1546895503997807
95% absolute error is 54.317835998534825, q-error is 5.3362582683563184
********Epoch 10, training loss: 9.684979233850715 || evaluation loss: ********


...

50% absolute error is 3.1073856353759766, q-error is 1.233727753162384
90% absolute error is 20.941141319274905, q-error is 2.882909965515137
95% absolute error is 33.386987304687416, q-error is 4.574425244331359
********Epoch 25, training loss: 5.824152775010897 || evaluation loss: ********


100%|█████████████████████████████████████████████████████████████████████████| 154/154 [00:04<00:00, 36.27it/s]


50% absolute error is 3.737357497215271, q-error is 1.26714426279068
90% absolute error is 25.326186370849637, q-error is 2.9937518358230593
95% absolute error is 40.391194152832014, q-error is 4.9079098701477
********Epoch 30, training loss: 8.527552574329977 || evaluation loss: ********


100%|█████████████████████████████████████████████████████████████████████████| 154/154 [00:04<00:00, 34.91it/s]


50% absolute error is 3.220808506011963, q-error is 1.2426883578300476
90% absolute error is 20.192880249023446, q-error is 2.933159375190735
95% absolute error is 33.325135231018024, q-error is 4.846180343627929


In [44]:
import pickle as pkl
with open("checkpoints/redshift_stage_model.pkl", "wb") as f:
    pkl.dump(ss, f)
rnn.save_model("checkpoints")

In [45]:
rnn2 = ConcurrentRNN(ss, 
                     "redshift",
                    input_size=len(ss.all_feature[0]) * 2 + 7,
                    embedding_dim=128,
                    hidden_size=256,
                    num_layers=2,
                    loss_function="q_loss",
                    last_output=True,
                    use_separation=False
                   )
rnn2.load_model("checkpoints")

In [46]:
preds, labels = rnn2.predict(eval_trace_df, use_pre_info_only=False)

100%|█████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 16.34it/s]

50% absolute error is 5.639205455780029, q-error is 1.1941273212432861
90% absolute error is 28.22308807373048, q-error is 2.0061857938766487
95% absolute error is 39.24708633422851, q-error is 2.3891061663627613


In [61]:
concurrency_df = create_concurrency_dataset(all_trace[0], engine=None, pre_exec_interval=200)
preds, labels = rnn2.predict(concurrency_df, use_pre_info_only=False)

100%|███████████████████████████████████████████████████████████████████████████| 42/42 [00:01<00:00, 23.03it/s]

50% absolute error is 1.6386100053787231, q-error is 1.2225273251533508
90% absolute error is 14.885701751708984, q-error is 2.4476436853408807
95% absolute error is 23.693497276306143, q-error is 3.6519856929779038


In [58]:
preds2, labels2 = rnn2.predict(eval_trace_df, use_pre_info_only=False)

100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 16.33it/s]

50% absolute error is 2.5743322372436523, q-error is 1.2749279141426086
90% absolute error is 18.791236114501956, q-error is 2.882374620437622
95% absolute error is 28.05649528503415, q-error is 4.55170130729675


In [48]:
i = 210
idx = np.argsort(preds2[i])
print(len(idx))
np.stack((np.asarray(preds2[i])[idx], np.asarray(preds[i])[idx], np.asarray(labels2[i])[idx]), axis=1)

111


array([[  2.8291447,   7.1109896,  79.540054 ],
       [  2.8653407,   2.3412442,   3.416112 ],
       [  2.934177 ,   4.0760264,   4.0271263],
       [  2.9910045,   3.7841144,   3.8717294],
       [  3.0511217,   5.0034904,   3.1730235],
       [  3.148857 ,  18.968218 ,   3.1449356],
       [  3.170916 ,   6.0808353, 106.32712  ],
       [  3.183551 ,   6.5285897,   4.7487164],
       [  3.2096725,  33.221123 ,   2.8220966],
       [  3.2253332,   5.26631  ,   3.3115742],
       [  3.246746 ,   4.4229527,   5.8701944],
       [  3.2892797,   5.2571554,   7.374345 ],
       [  3.2927618,  24.601357 ,   2.9005537],
       [  3.3001895,   3.752159 ,   4.9782295],
       [  3.3236983,   3.4375095,   4.804597 ],
       [  3.3453178,   4.3794765,   3.0326128],
       [  3.4195132,   5.3936315,   8.1688   ],
       [  3.4844189,   6.0924673,   4.3422008],
       [  3.5340056,   4.0386086,   4.554105 ],
       [  3.5757537,   8.585522 ,  32.302185 ],
       [  3.5846162,   6.08425  ,   5.50

In [8]:
i = 210
idx = np.argsort(preds2[i])
print(len(idx))
np.stack((np.asarray(preds[i])[idx], np.asarray(preds2[i])[idx], np.asarray(labels2[i])[idx]), axis=1)

NameError: name 'preds2' is not defined

In [11]:
i = 2
idx = np.argsort(preds[i])
print(len(idx))
np.stack((np.asarray(preds[i])[idx], np.asarray(labels[i])[idx]), axis=1)

7


array([[  9.086283,  10.558472],
       [  9.326768,  10.257236],
       [ 16.062849,  39.89623 ],
       [ 20.419281,  69.41913 ],
       [ 35.720085,  52.07332 ],
       [ 44.310814, 110.97227 ],
       [ 50.8906  ,  45.67449 ]], dtype=float32)

In [11]:
with open(directory + "postgres_mixed.sql", "r") as f:
    sql = f.readlines()

In [13]:
plans = load_json(parsed_queries_path, namespace=False)

In [16]:
aurora_sql = plans["sql_queries"]

In [15]:
plans.keys()

dict_keys(['parsed_plans', 'parsed_queries', 'sql_queries', 'database_stats', 'run_kwargs', 'skipped', 'blocks_accessed'])

In [21]:
all_index = []
for q in sql:
    i = aurora_sql.index(q.strip())
    all_index.append(i)

In [22]:
all_index

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186

In [23]:
sql[-1]

'SELECT AVG(imdb_id), AVG(kind_id), AVG(production_year) FROM title WHERE id < 42000000;'

In [25]:
sql[206]

'SELECT AVG(person_id), AVG(movie_id) FROM cast_info WHERE id < 25000000;\n'

In [26]:
aurora_sql[207]

'SELECT AVG(imdb_id), AVG(kind_id), AVG(production_year) FROM title WHERE id < 42000000;'

In [28]:
aurora_sql[-5:]

['SELECT AVG(movie_id), AVG(info_type_id) FROM movie_info WHERE id < 32000000;',
 'SELECT AVG(movie_id), AVG(keyword_id) FROM movie_keyword WHERE id < 42000000;',
 'SELECT AVG(imdb_id) FROM name WHERE id < 42000000;',
 'SELECT AVG(person_id) FROM person_info WHERE id < 42000000;',
 'SELECT AVG(imdb_id), AVG(kind_id), AVG(production_year) FROM title WHERE id < 42000000;']

In [29]:
postgres_plans = copy.deepcopy(plans)

In [30]:
postgres_plans['parsed_plans'] = [plans['parsed_plans'][i] for i in all_index]
postgres_plans['parsed_queries'] = [plans['parsed_queries'][i] for i in all_index]
postgres_plans['sql_queries'] = sql


In [39]:
import json
with open("/Users/ziniuw/Desktop/research/Data/AWS_trace/mixed_postgres/postgres_mixed_parsed_queries.json", "w") as f:
    json.dump(postgres_plans, f)